# Large-Scale Analysis of the Co-Commit Patterns of the Active Developers in GitHub’s Top Repositories (MSR'18)
## Part 1: Data Preparation
**Eldan Cohen, Mariano P. Consens**

## Imports and configurations

In [1]:
import glob
import os.path
from pyspark.sql.functions import *
import datetime
from functools import reduce
from collections import defaultdict

In [2]:
DATA_FOLDER = "gh_all"
OUTPUT_GRAPH_FOLDER = "gh_graphs"

## Loading GitHub's contribution data

DATA_FOLDER includes a CSV file per language covring the contribution data for the whole time period of three years

In [3]:
langDataPath = {os.path.basename(csvPath).split(".")[0].title(): csvPath
                for csvPath in glob.glob(os.path.join(DATA_FOLDER, "*"))}

In [4]:
langDataPath

{'C': 'gh_all/c.csv',
 'C#': 'gh_all/c#.csv',
 'C++': 'gh_all/c++.csv',
 'Clojure': 'gh_all/clojure.csv',
 'Go': 'gh_all/go.csv',
 'Haskell': 'gh_all/haskell.csv',
 'Java': 'gh_all/java.csv',
 'Javascript': 'gh_all/javascript.csv',
 'Julia': 'gh_all/julia.csv',
 'Ocaml': 'gh_all/ocaml.csv',
 'Php': 'gh_all/php.csv',
 'Python': 'gh_all/python.csv',
 'R': 'gh_all/r.csv',
 'Ruby': 'gh_all/ruby.csv',
 'Rust': 'gh_all/rust.csv',
 'Scala': 'gh_all/scala.csv',
 'Scheme': 'gh_all/scheme.csv'}

Read each CSV into a spark dataframe. langDF is a dict that maps from a language to the spark dataframe of its contribution data.

In [5]:
def readDF(lang, langPath):
    return spark.read.csv(langPath, header=True). \
        withColumn("language", lit(lang)). \
        withColumn("week", to_date(split("week", " ")[0], 'yyyy-MM-dd')). \
        withColumnRenamed("week", "date"). \
        selectExpr("repoName as repository",
                   "login",
                   "date",
                   "c as commits",
                   "language")

In [6]:
langDF = {lang: readDF(lang, langPath) for lang,langPath in langDataPath.items()}

Show the head of the dataframe:

In [7]:
langDF["Python"].limit(5).toPandas()

,repository,login,date,commits,language
0,jkbrzt/httpie,asnelzin,2014-11-09,1,Python
1,jkbrzt/httpie,dongweiming,2016-08-28,1,Python
2,jkbrzt/httpie,zquestz,2016-08-07,1,Python
3,jkbrzt/httpie,ii-v,2016-08-07,1,Python
4,jkbrzt/httpie,mmb,2012-04-01,1,Python


### Statistics per Language
Build a combined dataframe of all languages' data and compute some statistics.

In [8]:
combinedDF = reduce(lambda df1, df2: df1.union(df2), langDF.values())

In [9]:
combinedDF.groupby("language").agg(countDistinct("repository").alias("numRepositories"), 
                                   countDistinct("login").alias("numDevelopers"), 
                                   sum("commits").alias("numCommits")
                                  ).orderBy("language").toPandas()

,language,numRepositories,numDevelopers,numCommits
0,C,988,24980,1687163.0
1,C#,997,14863,838687.0
2,C++,995,27048,2155219.0
3,Clojure,997,3843,246797.0
4,Go,999,15214,623247.0
5,Haskell,993,3885,383923.0
6,Java,996,19404,1130188.0
7,Javascript,1000,39021,921785.0
8,Julia,993,1563,154033.0
9,Ocaml,967,1854,298147.0


## Creating Temporal Networks

First, we define the 10 time periods covering the whole three years

In [10]:
timePeriods = [
    (datetime.date(2015,10,1), datetime.date(2016,7,1)),  # t1
    (datetime.date(2015,7,1),  datetime.date(2016,4,1)),  # t2
    (datetime.date(2015,4,1),  datetime.date(2016,1,1)),  # t3
    (datetime.date(2015,1,1),  datetime.date(2015,10,1)), # t4
    (datetime.date(2014,10,1), datetime.date(2015,7,1)),  # t5
    (datetime.date(2014,7,1),  datetime.date(2015,4,1)),  # t6
    (datetime.date(2014,4,1),  datetime.date(2015,1,1)),  # t7
    (datetime.date(2014,1,1),  datetime.date(2014,10,1)), # t8
    (datetime.date(2013,10,1), datetime.date(2014,7,1)),  # t9
    (datetime.date(2013,7,1),  datetime.date(2014,4,1)),  # t10
]

The following function extract the full and core temporal networks for a given time period for a given language. 

*minDevelopers=2* eliminates private repositories, *theta=0.75* is the participation criteria. 

In [11]:
def extractTemporalNetworks(df, dateFrom, dateTo, minDevelopers = 2, theta = 0.75):
    contributionCount =  df.where(df.date >= dateFrom). \
                            where(df.date < dateTo). \
                            groupBy("language", "repository", "login"). \
                            agg(sum("commits").alias("loginCommits"))
    reposStats = contributionCount.groupBy("language", "repository"). \
                                   agg(avg("loginCommits").alias("avgContribution"), count("login").alias("loginCount"))
    
    filteredNetwork = contributionCount.join(reposStats, ["language", "repository"]). \
                                        where(reposStats.loginCount >= minDevelopers). \
                                        where(contributionCount.login.contains("invalid-email") == False). \
                                        where(contributionCount.login != "")
    
    fullNetwork = filteredNetwork.select("language", "repository", "login", "loginCommits")
    coreNetwork = filteredNetwork.where(filteredNetwork.loginCommits >= theta*filteredNetwork.avgContribution). \
                                  select("language", "repository", "login", "loginCommits")
    
    return (fullNetwork, coreNetwork)

Now, we compute all the temporal networks for each language for each of time periods. We also compute the temporal network for the cumulative time period (denoted $t_c$).

In [12]:
# Initialize a nested dict for temporal networks for each period and language: temporalNetworks[period][language]
temporalNetworks = defaultdict(dict)

for period in range(len(timePeriods)):
    # Get the start and end date of this time period
    periodFrom, periodTo = timePeriods[period]
    
    for lang in langDF:
        # Extract and save temporal network for the given period and language
        temporalNetworks[period+1][lang] = extractTemporalNetworks(langDF[lang], periodFrom, periodTo)
    
    # Report progress
    print("t%d" % (period+1,), end="; ")

# Extract and save cumulative network
cumFrom = timePeriods[-1][0] # start date of oldest time period
cumTo = timePeriods[0][1] # end date of most recent time period
for lang in langDF:
        temporalNetworks["c"][lang] = extractTemporalNetworks(langDF[lang], cumFrom, cumTo)
print("tc")

t1; t2; t3; t4; t5; t6; t7; t8; t9; t10; tc


### Statistics on the Temporal Networks

We present the fraction of active developers and percent of commits for both $t_1$ and $t_c$.

** Statistics for $t_1$: **

In [13]:
t1CoreAll = reduce(lambda df1, df2: df1.union(df2), [core for full, core in temporalNetworks[1].values()])
t1FullAll = reduce(lambda df1, df2: df1.union(df2), [full for full, core in temporalNetworks[1].values()])

In [14]:
t1CoreSummary = t1CoreAll.groupBy("language").agg(countDistinct("login").alias("numDevelopersCore"), 
                                                  sum("loginCommits").alias("numCommitsCore")).toPandas()
t1FullSummary = t1FullAll.groupBy("language").agg(countDistinct("login").alias("numDevelopersFull"), 
                                                  sum("loginCommits").alias("numCommitsFull")).toPandas()

In [15]:
t1Summary = t1CoreSummary.merge(t1FullSummary, on="language")
t1Summary["fractionActiveDevelopers"] = t1Summary["numDevelopersCore"]/t1Summary["numDevelopersFull"]
t1Summary["percentOfCommits"] = 100*(t1Summary["numCommitsCore"]/t1Summary["numCommitsFull"])
t1Summary = t1Summary[["language", "fractionActiveDevelopers", "percentOfCommits"]].sort_values("language").reset_index(drop=True)
t1Summary.loc['Median'] = t1Summary.median()
t1Summary.style.format({"fractionActiveDevelopers": "{:.3f}", "percentOfCommits": "{:.1f}"})

,language,fractionActiveDevelopers,percentOfCommits
0,C,0.226,87.0
1,C#,0.287,89.8
2,C++,0.240,89.3
3,Clojure,0.325,90.3
4,Go,0.240,89.6
5,Haskell,0.361,90.5
6,Java,0.242,88.7
7,Javascript,0.177,87.3
8,Julia,0.457,88.2
9,Ocaml,0.370,88.7


** Statistics for $t_c$: **

In [16]:
tcCoreAll = reduce(lambda df1, df2: df1.union(df2), [core for full, core in temporalNetworks["c"].values()])
tcFullAll = reduce(lambda df1, df2: df1.union(df2), [full for full, core in temporalNetworks["c"].values()])

In [17]:
tcCoreSummary = tcCoreAll.groupBy("language").agg(countDistinct("login").alias("numDevelopersCore"),
                                                  sum("loginCommits").alias("numCommitsCore")).toPandas()
tcFullSummary = tcFullAll.groupBy("language").agg(countDistinct("login").alias("numDevelopersFull"),
                                                  sum("loginCommits").alias("numCommitsFull")).toPandas()

In [18]:
tcSummary = tcCoreSummary.merge(tcFullSummary, on="language")
tcSummary["fractionActiveDevelopers"] = tcSummary["numDevelopersCore"]/tcSummary["numDevelopersFull"]
tcSummary["percentOfCommits"] = 100*(tcSummary["numCommitsCore"]/tcSummary["numCommitsFull"])
tcSummary = tcSummary[["language", "fractionActiveDevelopers", "percentOfCommits"]].sort_values("language").reset_index(drop=True)
tcSummary.loc['Median'] = tcSummary.median()
tcSummary.style.format({"fractionActiveDevelopers": "{:.3f}", "percentOfCommits": "{:.1f}"})

,language,fractionActiveDevelopers,percentOfCommits
0,C,0.173,89.5
1,C#,0.204,90.5
2,C++,0.182,91.3
3,Clojure,0.254,91.8
4,Go,0.177,90.3
5,Haskell,0.276,92.1
6,Java,0.169,90.8
7,Javascript,0.131,88.6
8,Julia,0.352,89.6
9,Ocaml,0.305,90.3


## Creating the graphs
We now create the graph files for each language for each time period.

These graph files will be analyzed according to the chosen metrics in the following notebook.

- *graphDF* takes a temporal network created by *extractTemporalNetworks* and build a dataframe that represents the corresponding graph (list of edges)

In [19]:
def graphDF(df):
    # Select only the required fields
    dfSelected = df.select("repository", "login")
    
    # Create a dataframe of the edges in the graph
    return dfSelected.withColumnRenamed("login", "login1"). \
                      join(dfSelected.withColumnRenamed("login", "login2"), "repository"). \
                      where("login1 <= login2"). \
                      select("login1", "login2"). \
                      distinct()

* Now, we iterate over all time periods and languages and for each crate csv files for the corresponding full network graph and core network graph

In [20]:
for period in temporalNetworks:
    
    # Create a subfolder for the graphs of this time period in OUTPUT_GRAPH_FOLDER
    graphFolderPath = os.path.join(OUTPUT_GRAPH_FOLDER, 
                                       "g%s" % (str(period),))
    os.makedirs(graphFolderPath, exist_ok=True)
    
    # Report progress
    print("t%s" % (str(period),), end=": ")
    
    for lang in temporalNetworks[period]:
        
        # Extract the temporal core and full network for the language in the current time period
        tempFull, tempCore = temporalNetworks[period][lang]
        
        # Write a CSV of the core graph edges
        corePath = os.path.join(graphFolderPath,
                                "%s_core.csv" % (lang,))
        graphDF(tempCore).toPandas().to_csv(corePath, index=False, header=False)
        
        # Write a CSV of the full graph edges
        fullPath = os.path.join(graphFolderPath,
                                "%s_full.csv" %(lang,))
        graphDF(tempFull).toPandas().to_csv(fullPath, index=False, header=False)
        
        # Report progress
        print("%s" % (lang,), end=", ")
    
    print() # New line
    

t1: Ruby, Php, Scheme, Python, Clojure, C, Ocaml, C#, Javascript, Rust, R, Go, C++, Scala, Haskell, Java, Julia, 
t2: Ruby, Php, Scheme, Python, Clojure, C, Ocaml, C#, Javascript, Rust, R, Go, C++, Scala, Haskell, Java, Julia, 
t3: Ruby, Php, Scheme, Python, Clojure, C, Ocaml, C#, Javascript, Rust, R, Go, C++, Scala, Haskell, Java, Julia, 
t4: Ruby, Php, Scheme, Python, Clojure, C, Ocaml, C#, Javascript, Rust, R, Go, C++, Scala, Haskell, Java, Julia, 
t5: Ruby, Php, Scheme, Python, Clojure, C, Ocaml, C#, Javascript, Rust, R, Go, C++, Scala, Haskell, Java, Julia, 
t6: Ruby, Php, Scheme, Python, Clojure, C, Ocaml, C#, Javascript, Rust, R, Go, C++, Scala, Haskell, Java, Julia, 
t7: Ruby, Php, Scheme, Python, Clojure, C, Ocaml, C#, Javascript, Rust, R, Go, C++, Scala, Haskell, Java, Julia, 
t8: Ruby, Php, Scheme, Python, Clojure, C, Ocaml, C#, Javascript, Rust, R, Go, C++, Scala, Haskell, Java, Julia, 
t9: Ruby, Php, Scheme, Python, Clojure, C, Ocaml, C#, Javascript, Rust, R, Go, C++, Scal

Verify graph files were created:

In [23]:
!ls -la gh_graphs/g1

total 148876
drwxrwxr-x  2 my_user my_user     4096 May 21 15:51 .
drwxrwxr-x 13 my_user my_user     4096 May 21 16:19 ..
-rw-rw-r--  1 my_user my_user  1141150 May 21 15:49 C_core.csv
-rw-rw-r--  1 my_user my_user   154687 May 21 15:50 C#_core.csv
-rw-rw-r--  1 my_user my_user   315350 May 21 15:51 C++_core.csv
-rw-rw-r--  1 my_user my_user 37133645 May 21 15:50 C_full.csv
-rw-rw-r--  1 my_user my_user  2508478 May 21 15:50 C#_full.csv
-rw-rw-r--  1 my_user my_user  8248483 May 21 15:51 C++_full.csv
-rw-rw-r--  1 my_user my_user    15800 May 21 15:49 Clojure_core.csv
-rw-rw-r--  1 my_user my_user   190792 May 21 15:49 Clojure_full.csv
-rw-rw-r--  1 my_user my_user   224717 May 21 15:50 Go_core.csv
-rw-rw-r--  1 my_user my_user  8681102 May 21 15:51 Go_full.csv
-rw-rw-r--  1 my_user my_user    24812 May 21 15:51 Haskell_core.csv
-rw-rw-r--  1 my_user my_user   421505 May 21 15:51 Haskell_full.csv
-rw-rw-r--  1 my_user my_user   239597 May 21 15:51 Java_core.csv
-rw-rw-r--  1 my_user my